# 125个经典技术指标择时分析
## 广发证券 2020年1月3日

In [4]:
# 数据接口 
import akshare as ak
import baostock as bs
import tushare as ts

# 基础模块
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import os

# 回测框架
import backtrader as bt
import backtrader.indicators as btind
import backtrader.feeds as btfeed
import indicatorJ

# 基础函数
import utilsJ

## 策略

### 新因子测试

In [7]:
class gf125_test(bt.Strategy):
    
    params = (
        ('printlog', False),
    )
    
    
    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.data.datetime.date(0)
            print('%s: %s' % (dt.isoformat(), txt))
            #with open('log.txt', 'a') as file:
                #file.write('%s: %s \n' % (dt.isoformat(), txt))
        
    
    def __init__(self):
        
        # Keep a reference to buyprice and sellprice in the data[0] dataseries
        self.buyprice = None
        self.sellprice = None
        
        # Add indicators
        self.PAC = indicatorJ.PAC(self.data)


    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                
                self.log('BUY EXECUTED, Price: %.2f, Lot:%i, Cash: %i, Value: %i' %
                         (order.executed.price,
                          order.executed.size,
                          self.broker.get_cash(),
                          self.broker.get_value()))
                self.buyprice = order.executed.price

            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Lot:%i, Cash: %i, Value: %i' %
                        (order.executed.price,
                          -order.executed.size,
                          self.broker.get_cash(),
                          self.broker.get_value()))
                self.sellprice = order.executed.price

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')


    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
        
        
    def next(self):
        # if self.order:
        #     return
        print(self.data.datetime.date(0))
        print(self.data.high[-1])
        print(self.data.high[0])
        print(self.PAC.lines.Upper[0])

### 策略主体

In [7]:
class gf125(bt.Strategy):
    
    params = (
        ('printlog', False),
        ('N1', 1), 
        ('N2', 5),
        ('N3', 10),
    )
    
    
    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.data.datetime.date(0)
            print('%s: %s' % (dt.isoformat(), txt))
            #with open('log.txt', 'a') as file:
                #file.write('%s: %s \n' % (dt.isoformat(), txt))
        
    
    def __init__(self):
        
        # Keep a reference to buyprice and sellprice in the data[0] dataseries
        self.buyprice = None
        self.sellprice = None
        
        # Add indicators
        ## 价格动量类因子
        self.ER = indicatorJ.ER(self.data, plot=False)
        self.DPO = indicatorJ.DPO(self.data, plot=False)
        self.POS = indicatorJ.POS(self.data, plot=False)
        self.TII = indicatorJ.TII(self.data, plot=False)
        self.ADTM = indicatorJ.ADTM(self.data, plot=False)
        self.MADisplaced = indicatorJ.MADisplaced(self.data, plot=False)
        self.T3 = indicatorJ.T3(self.data, plot=False)
        self.VMA = indicatorJ.VMA(self.data, plot=False)
        self.BIAS = indicatorJ.BIAS(self.data, plot=False)
        self.TMA = indicatorJ.TMA(self.data, plot=False)
        self.TYP = indicatorJ.TYP(self.data, plot=False)
        self.WMA_BS = indicatorJ.WMA_BS(self.data, plot=False)
        self.PAC = indicatorJ.PAC(self.data, plot=False)
        self.MTM = indicatorJ.MTM(self.data, plot=False)

        ## 价格翻转类
        self.KDJ = indicatorJ.KDJ(self.data, plot=False)
        self.RMI = indicatorJ.RMI(self.data, plot=False)
        self.SKDJ = indicatorJ.SKDJ(self.data, plot=False)
        self.CCI = indicatorJ.CCI(self.data, plot=False)

        ## 成交量类
        self.MAAMT = indicatorJ.MAAMT(self.data, plot=False)
        self.SROCVOL = indicatorJ.SROCVOL(self.data, plot=False)

        # Add trend indicator
        self.uptrend = bt.And(self.data.low == btind.Highest(self.data.low, period=self.p.N3), 
                              self.data.high == btind.Highest(self.data.high, period=self.p.N3))
        

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                
                self.log('BUY EXECUTED, Price: %.2f, Lot:%i, Cash: %i, Value: %i' %
                         (order.executed.price,
                          order.executed.size,
                          self.broker.get_cash(),
                          self.broker.get_value()))
                self.buyprice = order.executed.price

            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Lot:%i, Cash: %i, Value: %i' %
                        (order.executed.price,
                          -order.executed.size,
                          self.broker.get_cash(),
                          self.broker.get_value()))
                self.sellprice = order.executed.price

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')


    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
        
        
    def next(self):
        # if self.order:
        #    return

        # Major votes
        class1_buy = [self.ER.lines.Buy[0], self.DPO.lines.Buy[0], self.POS.lines.Buy[0], self.TII.lines.Buy[0],
                      self.ADTM.lines.Buy[0], self.MADisplaced.lines.Buy[0], self.T3.lines.Buy[0], self.VMA.lines.Buy[0],
                      self.BIAS.lines.Buy[0], self.TMA.lines.Buy[0], self.TYP.lines.Buy[0], self.WMA_BS.lines.Buy[0],
                      self.PAC.lines.Buy[0], self.MTM.lines.Buy[0]]
        class2_buy = [self.KDJ.lines.Buy[0], self.RMI.lines.Buy[0], self.SKDJ.lines.Buy[0], self.CCI.lines.Buy[0]]
        class3_buy = [self.MAAMT.lines.Buy[0], self.SROCVOL.lines.Buy[0]]

        class1_sell = [self.ER.lines.Sell[0], self.DPO.lines.Sell[0], self.POS.lines.Sell[0], self.TII.lines.Sell[0],
                       self.ADTM.lines.Sell[0], self.MADisplaced.lines.Sell[0], self.T3.lines.Sell[0], self.VMA.lines.Sell[0],
                       self.BIAS.lines.Sell[0], self.TMA.lines.Sell[0], self.TYP.lines.Sell[0], self.WMA_BS.lines.Sell[0],
                       self.PAC.lines.Sell[0], self.MTM.lines.Sell[0]]
        class2_sell = [self.KDJ.lines.Sell[0], self.RMI.lines.Sell[0], self.SKDJ.lines.Sell[0], self.CCI.lines.Sell[0]]
        class3_sell = [self.MAAMT.lines.Sell[0], self.SROCVOL.lines.Sell[0]]

        major_vote_buy = max(class1_buy, key=class1_buy.count) + max(class2_buy, key=class2_buy.count) + max(class3_buy, key=class3_buy.count)
        major_vote_sell = max(class1_sell, key=class1_sell.count) + max(class2_sell, key=class2_sell.count) + max(class3_sell, key=class3_sell.count)
        

        if major_vote_buy > 0 and self.getposition(self.data).size == 0 and self.uptrend[0]:
            lots = np.floor(self.broker.getcash() / (100*self.data.close[0]))*100
            self.log('BUY CREATE, Price: %.2f, Lots: %i, Current Position: %i' % (self.data.close[0], lots, self.getposition(self.data).size))
            self.order = self.buy(size=lots)
        elif major_vote_sell > 0 and self.getposition(self.data).size != 0 and self.uptrend[0]:
            self.log('Sell CREATE (Close), Price: %.2f, Current Position: %i' % (self.data.close[0], self.getposition(self.data).size))
            self.order = self.close()

## 新因子测试

In [ ]:
s_date = datetime.date(2019,12,31)
e_date = datetime.date(2022,1,1)
stock_code = '600588.SH'
token = '74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6'


if __name__ ==  '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    strats = cerebro.addstrategy(gf125_test, printlog=True)


    # Create stock Data Feed
    if os.path.exists('.\\Data\\'+stock_code+'.csv'):
        df = pd.read_csv('.\\Data\\'+stock_code+'.csv', 
                    converters={'trade_date':lambda x:pd.to_datetime(x)}).set_index('trade_date')
    else:
        df = utilsJ.stock_tushare(token, stock_code, s_date, e_date)

    data = btfeed.PandasData(dataname=df,fromdate=s_date,todate=e_date)

    # Add the index Data Feed to Cerebo
    cerebro.adddata(data)

    # Set cash inside the strategy
    cerebro.broker = bt.brokers.BackBroker(coc=True)   
    cerebro.broker.setcash(200000)

    # Set commission
    #cerebro.broker.setcommission()

    # Print out the starting conditions
    start_value = cerebro.broker.getvalue()
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    final_value = cerebro.broker.getvalue()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    print('Net Profit: %.2f%%' % ((final_value - start_value) / start_value * 100))

    # Visualization
    #cerebro.plot(iplot=False)

## 单股回测

In [12]:
s_date = datetime.date(2019,6,30)
e_date = datetime.date(2022,10,28)
stock_code = '000005.SZ'
token = '74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6'


if __name__ ==  '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    strats = cerebro.addstrategy(gf125, printlog=True)


    # Create stock Data Feed
    # if os.path.exists('.\\Data\\'+stock_code+'.csv'):
    #     df = pd.read_csv('.\\Data\\'+stock_code+'.csv', 
    #                 converters={'trade_date':lambda x:pd.to_datetime(x)}).set_index('trade_date')
    # else:
    #     df = utilsJ.stock_tushare(token, stock_code, s_date, e_date)
    df = utilsJ.stock_tushare(token, stock_code, s_date, e_date)
    data = btfeed.PandasData(dataname=df,fromdate=s_date,todate=e_date)

    # Add the index Data Feed to Cerebo
    cerebro.adddata(data)

    # Set cash inside the strategy
    cerebro.broker = bt.brokers.BackBroker(coc=True)   
    cerebro.broker.setcash(200000)

    # Set commission
    #cerebro.broker.setcommission()

    # Print out the starting conditions
    start_value = cerebro.broker.getvalue()
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    final_value = cerebro.broker.getvalue()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    print('Net Profit: %.2f%%' % ((final_value - start_value) / start_value * 100))

    # Visualization
    cerebro.plot(iplot=False)

Starting Portfolio Value: 200000.00
2020-11-23: BUY CREATE, Price: 2.70, Lots: 74000, Current Position: 0
2020-11-24: BUY EXECUTED, Price: 2.70, Lot:74000, Cash: 200, Value: 198520
2021-06-04: Sell CREATE (Close), Price: 1.83, Current Position: 74000
2021-06-07: SELL EXECUTED, Price: 1.83, Lot:74000, Cash: 135620, Value: 135620
2021-06-07: OPERATION PROFIT, GROSS -64380.00, NET -64380.00
2021-06-24: BUY CREATE, Price: 1.92, Lots: 70600, Current Position: 0
2021-06-25: BUY EXECUTED, Price: 1.92, Lot:70600, Cash: 68, Value: 137032
2021-08-31: Sell CREATE (Close), Price: 2.39, Current Position: 70600
2021-09-01: SELL EXECUTED, Price: 2.39, Lot:70600, Cash: 168802, Value: 168802
2021-09-01: OPERATION PROFIT, GROSS 33182.00, NET 33182.00
2021-10-15: BUY CREATE, Price: 2.28, Lots: 74000, Current Position: 0
2021-10-18: BUY EXECUTED, Price: 2.28, Lot:74000, Cash: 82, Value: 164362
2022-03-02: Sell CREATE (Close), Price: 2.28, Current Position: 74000
2022-03-03: SELL EXECUTED, Price: 2.28, Lot

# 沪深300/全市场回测

In [13]:
s_date = datetime.date(2019,12,31)
e_date = datetime.date(2022,10,28)
stock_index = '000300.SH'
token = '74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6'

pro = ts.pro_api(token)
#index_list = np.unique(pro.index_weight(index_code=stock_index, 
#                          start_date=s_date.strftime('%Y%m%d'), 
#                          end_date=e_date.strftime('%Y%m%d')).con_code).tolist()
index_list = np.unique(pro.query('stock_basic', exchange='', list_status='L', fields='ts_code')).tolist()
profit_dict = {}

if __name__ ==  '__main__':

    for stock_code in index_list:
        print(stock_code)
        try:
            # Create a cerebro entity
            cerebro = bt.Cerebro()

            # Add a strategy
            strats = cerebro.addstrategy(gf125, printlog=True)

            # Create stock Data Feed
            if os.path.exists('C:\\Users\\a\\Desktop\\Backtrader\\Data\\'+stock_code+'.csv'):
                df = pd.read_csv('C:\\Users\\a\\Desktop\\Backtrader\\Data\\'+stock_code+'.csv', 
                            converters={'trade_date':lambda x:pd.to_datetime(x)}).set_index('trade_date')
            else:
                df = utilsJ.stock_tushare(token, stock_code, s_date, e_date)

            data = btfeed.PandasData(dataname=df,fromdate=s_date,todate=e_date)

            # Add the index Data Feed to Cerebo
            cerebro.adddata(data)

            # Set cash inside the strategy
            cerebro.broker = bt.brokers.BackBroker(coc=True)   
            cerebro.broker.setcash(2000000)

            # Set commission
            #cerebro.broker.setcommission()

            # Print out the starting conditions
            start_value = cerebro.broker.getvalue()
            #print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

            # Run over everything
            cerebro.run()

            # Print out the final result
            final_value = cerebro.broker.getvalue()
            #print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
            print('Net Profit: %.2f%%' % ((final_value - start_value) / start_value * 100))
            profit_dict[stock_code] = (final_value - start_value) / start_value * 100
        except:
            print('failed.')

000001.SZ
2020-11-23: BUY CREATE, Price: 19.12, Lots: 104500, Current Position: 0
2020-11-24: BUY EXECUTED, Price: 19.12, Lot:104500, Cash: 1667, Value: 1973519
2020-11-24: Sell CREATE (Close), Price: 18.87, Current Position: 104500
2020-11-25: SELL EXECUTED, Price: 18.87, Lot:104500, Cash: 1973519, Value: 1973519
2020-11-25: OPERATION PROFIT, GROSS -26480.30, NET -26480.30
2020-11-30: BUY CREATE, Price: 19.24, Lots: 102500, Current Position: 0
2020-12-01: BUY EXECUTED, Price: 19.24, Lot:102500, Cash: 1440, Value: 2004484
2021-02-05: Sell CREATE (Close), Price: 24.30, Current Position: 102500
2021-02-08: SELL EXECUTED, Price: 24.30, Lot:102500, Cash: 2492015, Value: 2492015
2021-02-08: OPERATION PROFIT, GROSS 518496.25, NET 518496.25
2021-04-21: BUY CREATE, Price: 22.43, Lots: 111100, Current Position: 0
2021-04-22: BUY EXECUTED, Price: 22.43, Lot:111100, Cash: 387, Value: 2488771
2021-04-22: Sell CREATE (Close), Price: 22.40, Current Position: 111100
2021-04-23: SELL EXECUTED, Price: 